# This class is using the angle-continuity algorithm to assign cell masks to chains #

In [1]:
from chain_detector import loadAndInitialize, masksToGraph, dropShortConnectedComponent, detectLeaves, detectChains, loadPkl, manuelChainCorrection, removeDuplicateChains, removeEmptyChains, showNapariWithColoredChainsAndAddChainFromScratch, assurePositionsInChains, saveListOfGraphsToPklAndChains3D
from focus_differentiator import getRegionProps
#import os

In [2]:
LOAD_CHAINS_GRAPHS = False
FOV = 0
HYB = 0
Z_SPAN = 7
XY_SPAN = 2
ANGLE_EPS = 85.0
ANGLE_DIFF_TIE_BREAKER = 65.0
PIXEL_THRESH = 9
CHAIN_LENGTH_LOWER_BOUND = 1
BEGIN_MANUAL_CORRECTION_FROM_CHAIN = 50
END_MANUAL_CHAIN_CORRECTION_AT_CHAIN = 52
IMG_SLICE = [0, -1, 0, -1]
#IMG_SLICE = [300, 600, 300, 600]
FILE_NAME = f'fov_{FOV}_hyb_{HYB}'
FOCUS_MASKS_PATH = rf'\fov_{FOV}_hyb_{HYB}\{FILE_NAME}.seg.npy'
CELL_PROPS_PATH = f'{FILE_NAME}\\{FILE_NAME}.props.txt'
DATA_DIR = r'Z:\sandbox\for_vadim\nostoc_auto_110924'
LIST_OF_CHAIN_GRAPHS_FILE_NAME = f'{FILE_NAME}\\fov_{FOV}_hyb_{HYB}.list_of_chain_Graphs.pkl'
CHAINS_AS_3D_FILENAME = f'fov_{FOV}_hyb_{HYB}.chains_as_3d_array'

# Running the angle-continuity algorithm, or loading
1. Change dir to current FOV we are working on.
2. If LOAD_CHAINS_GRAPHS is False:
    Load data structures: phase image, focus masks calculated in previous notebook, and dimensions
    Convert the image masks to Graph representation
    find the leaves (the chain tips, cell masks with one neighbor)
    Run the angle continuity algorith
3. If LOAD_CHAINS_GRAPHS is True:
    Load the images and the chains from file

In [3]:
#os.chdir(f'C:\\Users\\LITVINOV\\PycharmProjects\\anabaenaSeg\\fov_{FOV}_hyb_{HYB}')
#full_path_for_graphs = f'{FILE_NAME}\\' + LIST_OF_CHAIN_GRAPHS_FILE_NAME
if not LOAD_CHAINS_GRAPHS:
    img, masks, z_dim, r_dim, c_dim, adjacency_g = loadAndInitialize(DATA_DIR, FILE_NAME, FOCUS_MASKS_PATH, IMG_SLICE)
        #masks = masks[:, IMG_SLICE[0]:IMG_SLICE[1], IMG_SLICE[2]:IMG_SLICE[3]]  # z (depth),c (channel),x,y
        # Only phase
    img = img[:, 0, :, :]
    adjacency_g = masksToGraph(masks, adjacency_g, r_dim, c_dim, z_dim, z_span=Z_SPAN, xy_span=XY_SPAN, pixel_thresh=PIXEL_THRESH)
    adjacency_g = dropShortConnectedComponent(adjacency_g, CHAIN_LENGTH_LOWER_BOUND)
    leaves = detectLeaves(adjacency_g)
    chains_g = detectChains(masks, leaves, adjacency_g, ANGLE_EPS, ANGLE_DIFF_TIE_BREAKER)

    saveListOfGraphsToPklAndChains3D(chains_g, None, LIST_OF_CHAIN_GRAPHS_FILE_NAME, CHAINS_AS_3D_FILENAME)
else:
    img, masks, _, _, _, _ = loadAndInitialize(DATA_DIR, FILE_NAME, FOCUS_MASKS_PATH, IMG_SLICE)
    img = img[:, 0, :, :]
    chains_g = loadPkl(LIST_OF_CHAIN_GRAPHS_FILE_NAME)

Loading image and segmentation masks...


Converting to Graph representation...: 100%|██████████| 17/17 [00:00<00:00, 25.90it/s]


Dropping non-chains connected components...
Using angle continuity algorithm for detecting chains.
330 leaves remaining...
328 leaves remaining...
326 leaves remaining...
325 leaves remaining...
323 leaves remaining...
322 leaves remaining...
320 leaves remaining...
318 leaves remaining...
318 leaves remaining...
316 leaves remaining...
314 leaves remaining...
315 leaves remaining...
314 leaves remaining...
312 leaves remaining...
310 leaves remaining...
310 leaves remaining...
308 leaves remaining...
306 leaves remaining...
305 leaves remaining...
303 leaves remaining...
301 leaves remaining...
299 leaves remaining...
297 leaves remaining...
296 leaves remaining...
294 leaves remaining...
292 leaves remaining...
291 leaves remaining...
293 leaves remaining...
291 leaves remaining...
289 leaves remaining...
287 leaves remaining...
285 leaves remaining...
283 leaves remaining...
284 leaves remaining...
282 leaves remaining...
280 leaves remaining...
278 leaves remaining...
276 leaves re

# Manually correct the chain assignment if needed
The function loops over all chains from "BEGIN_..." to "END_..." parametres and correcting them manually (adding and dropping masks)

In [4]:
for chain_n in range(BEGIN_MANUAL_CORRECTION_FROM_CHAIN, END_MANUAL_CHAIN_CORRECTION_AT_CHAIN):
    chain = chains_g[chain_n]
    chains_g[chain_n] = manuelChainCorrection(chain, masks, img, BEGIN_MANUAL_CORRECTION_FROM_CHAIN, END_MANUAL_CHAIN_CORRECTION_AT_CHAIN, LIST_OF_CHAIN_GRAPHS_FILE_NAME, chain_n)

<class 'numpy.ndarray'>


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


# Check if there are any duplicate or empty chains

In [5]:
chains_g, assigned_masks = removeDuplicateChains(chains_g)
chains_g = removeEmptyChains(chains_g)

Checking that each node appears only in one chain: 100%|██████████| 199/199 [00:00<?, ?it/s]

Number of chains before removing duplicated and empty chains: 199
Number of chains after removing duplicated and empty chains: 199


# Show all chains with Napari, and assure that the relative positioning of cells in each chain are reliable
The loading can take a while, and the Napari window will be shown frozen.
For assuring the positioning, we need to load cell region_props that was created in the focus_differentiator step

In [6]:
chains_g, all_chains_3d, = showNapariWithColoredChainsAndAddChainFromScratch(img, chains_g, masks, assigned_masks)
props = getRegionProps(None, file_name=CELL_PROPS_PATH, load=True)
chains_g = assurePositionsInChains(chains_g, props)

Showing chains with Napari: 100%|██████████| 199/199 [01:23<00:00,  2.37it/s]


All cell positions in chain are consistent with centroid locations.


# Save the chains assignment as Graphs

In [7]:
saveListOfGraphsToPklAndChains3D(chains_g, all_chains_3d, LIST_OF_CHAIN_GRAPHS_FILE_NAME, CHAINS_AS_3D_FILENAME)


Saving chains in Graph list and as 3D array...
Chains are saved in .pkl file
